# Assess snowline altitude uncertainty

In [1]:
import os
import glob
import pandas as pd
import xarray as xr
import rioxarray as rxr
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
from shapely.geometry import box

In [ ]:
# Define path to snow cover data and site names
scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping'
rgi_ids = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(scm_path, 'study-sites', 'RGI*')))]
rgi_ids

## Define function for calculating SLA lower and upper bounds

The original SLA was calculated by sampling the $1-AAR$ percentile of the DEM. For example, if the AAR is 0.8, the SLA is calculated as the 20th percentile of elevations over the glacier area. 

$P_{SLA} = 1-AAR$

$SLA = P_{SLA}(DEM)$

To estimate upper and lower bounds for SLA, identify "misclassified" pixels above and below the SLA, and use those to adjust the SLA percentile. 

For the upper bound, calculate the area of snow-free pixels above the SLA, convert that to a percentile relative to the total area, and add that to the original SLA percentile. Sample the $P_{upper}$ of the DEM.  

$P_{upper} = \frac{A_{snow free, above SLA}}{A_{glacier}} + P_{SLA}$

$SLA_{upper} = P_{upper}(DEM)$

For the lower bound, calculate the area of snow-covered pixels below the SLA, convert that to a percentile relative to the total area, and subtract that from the original SLA percentile. Sample the $P_{lower}$ of the DEM. 

$P_{lower} = -\frac{A_{snow covered, below SLA}}{A_{glacier}} + P_{SLA}$

$SLA_{lower} = P_{lower}(DEM)$


In [3]:
def calculate_sla_bounds(stats_df, dem, snow_cover_mask, dx, verbose=False):
    # Get original snow cover stats and SLA percentile from DataFrame
    aar = stats_df['AAR']
    sla = stats_df['ELA_from_AAR_m']
    sla_percentile = 1 - aar

    # Calculate glacier area
    npx = len(np.argwhere(~np.isnan(dem.data.ravel())).ravel())
    total_area = npx * dx**2
        
    # Calculate areas of misclassified pixels
    snow_free_above_sla = xr.where((dem > sla) & (snow_cover_mask == 0), 1, 0)
    snow_free_above_sla_area = len(np.argwhere(snow_free_above_sla.data.ravel()==1).ravel()) * dx**2
    snow_covered_below_sla = xr.where((dem < sla) & (snow_cover_mask == 1), 1, 0)
    snow_covered_below_sla_area = len(np.argwhere(snow_covered_below_sla.data.ravel()==1).ravel()) * dx**2

    # Convert areas to percentiles
    delta_up = snow_free_above_sla_area / total_area
    delta_down = snow_covered_below_sla_area / total_area
        
    # Adjust SLA percentiles
    upper_sla_percentile = sla_percentile + delta_up
    lower_sla_percentile = sla_percentile - delta_down
    # Make sure percentiles are within [0,1]
    upper_sla_percentile, lower_sla_percentile = np.clip([upper_sla_percentile, lower_sla_percentile], 0, 1)

    # Calculate SLA upper and lower bounds
    sla_upper_bound = np.nanpercentile(dem.data.ravel(), upper_sla_percentile * 100)
    sla_lower_bound = np.nanpercentile(dem.data.ravel(), lower_sla_percentile * 100)

    # Print results if requested
    if verbose:
        print(f"Total area = {np.round(total_area / 1e6,2)} km^2")
        print(f"Area of snow-free pixels above SLA = {np.round(snow_free_above_sla_area / 1e6, 2)} km^2")
        print(f"Area of snow-covered pixels below SLA = {np.round(snow_covered_below_sla_area / 1e6, 2)} km^2")
        print(f"AAR = {np.round(aar, 3)}")
        print(f"SLA percentile = {np.round(sla_percentile, 3)}")
        print(f"SLA upper-bound percentile = {np.round(upper_sla_percentile, 3)}")
        print(f"SLA lower-bound percentile = {np.round(lower_sla_percentile, 3)}")
        print(f"SLA = {np.round(sla, 2)} m")
        print(f"SLA upper-bound = {np.round(sla_upper_bound, 2)} m")
        print(f"SLA lower-bound = {np.round(sla_lower_bound, 2)} m")
        
    return sla, sla_upper_bound, sla_lower_bound

## Calculate SLA bounds (uncertainty) for all sites and classified images

In [ ]:
# Define output file name
sla_bounds_fn = os.path.join(scm_path, 'analysis', 'SLA_uncertainty_analysis.csv')
if not os.path.exists(sla_bounds_fn):
    # Initialize results DataFrame
    sla_bounds_df = pd.DataFrame()

    # Iterate over sites
    for rgi_id in tqdm(rgi_ids[0:1]):
        # Load snow cover stats
        scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_snow_cover_stats.csv")
        # skip if snow cover stats or classified images do not exist
        if not os.path.exists(scs_fn):
            continue
        if not os.path.exists(os.path.join(scm_path, 'study-sites', rgi_id, 'classified')):
            continue
        scs = pd.read_csv(scs_fn)
        # remove rows where AAR = 1
        scs = scs.loc[scs['AAR'] < 1].reset_index(drop=True)
        
        # Load DEM
        dem_fn = glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'DEMs', '*.tif'))[0]
        dem = rxr.open_rasterio(dem_fn).isel(band=0)
        dem = xr.where((dem < -1e3) | (dem > 1e4), np.nan, dem)
        dem = dem.rio.write_crs("EPSG:4326")
        
        # Load AOI
        aoi_fn = os.path.join(scm_path, 'study-sites', rgi_id, 'AOIs', f"{rgi_id}_outline.shp")
        aoi = gpd.read_file(aoi_fn)
        
        # Clip DEM to AOI
        dem = dem.rio.clip(aoi.geometry)
    
        # Iterate over snow cover observations
        sla_originals = np.zeros(len(scs))
        sla_lower_bounds = np.zeros(len(scs))
        sla_upper_bounds = np.zeros(len(scs))
        dts = [''] * len(scs)
        sources = [''] * len(scs)
        for i in range(len(scs)):
            sc = scs.iloc[i]
            # Load classified image file
            dts[i] = sc['datetime']
            sources[i] = sc['source']
            classified_fn = glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'classified', 
                                                   f"{dts[i].replace('-', '').replace(':','')}_{rgi_id}_{sources[i]}_classified.nc"))[0]
            classified = rxr.open_rasterio(classified_fn).squeeze()
            classified = xr.where(classified==-9999, np.nan, classified)
            classified = classified.rio.write_crs("EPSG:4326")
            
            # Create binary snow image
            snow_binary = xr.where((classified==1) | (classified==2), 1, 0)
            snow_binary = xr.where(np.isnan(classified), np.nan, snow_binary) # re-insert no data values
            
            # Regrid DEM to classified image grid
            dem_adj = dem.rio.reproject_match(classified)
            dem_adj = xr.where(dem_adj > 1e4, np.nan, dem_adj)
            dem_adj = dem_adj.rio.write_crs("EPSG:4326")
            
            # Determine spatial resolution based on source
            if sources[i]=='Landsat':
                dx = 30
            elif 'Sentinel-2' in sources[i]:
                dx = 10
            
            # Calculate lower and upper bounds of snowline altitude
            sla_originals[i], sla_lower_bounds[i], sla_upper_bounds[i] = calculate_sla_bounds(sc, dem_adj, snow_binary, dx=dx, verbose=False)
            
        # Save in DataFrame
        df = pd.DataFrame({'RGIId': [rgi_id] * len(scs),
                           'datetime': dts,
                           'source': sources,
                           'SLA_m': sla_originals,
                           'SLA_lower_bound_m': sla_lower_bounds,
                           'SLA_upper_bound_m': sla_upper_bounds})
        
        # Concatenate to results DataFrame
        sla_bounds_df = pd.concat([sla_bounds_df, df], axis=0)
    
    # Save results to file
    sla_bounds_df.reset_index(drop=True, inplace=True)
    sla_bounds_df.to_csv(sla_bounds_fn, index=False)
    print('SLA bounds saved to file:', sla_bounds_fn)
    
else:
    sla_bounds_df = pd.read_csv(sla_bounds_fn)

# Add column for total range and describe stats
sla_bounds_df['SLA_bounds_range_m'] = np.abs(sla_bounds_df['SLA_upper_bound_m'] - sla_bounds_df['SLA_lower_bound_m'])
plt.boxplot(sla_bounds_df['SLA_bounds_range_m'], showfliers=False)
plt.show()

sla_bounds_df['SLA_bounds_range_m'].describe()    

## Plot results with an example calculation for supporting information

In [ ]:
# Example site ID
rgi_id = 'RGI60-01.01104'
epsg_utm = "EPSG:32606"

# Load all AOIs
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
# Add elevation range column
aois['Zrange'] = aois['Zmax'] - aois['Zmin']
# Merge with SLA bounds
sla_bounds_df = sla_bounds_df.merge(aois[['RGIId', 'Zrange']], on='RGIId')

# Load snow cover stats
scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_snow_cover_stats.csv")
scs = pd.read_csv(scs_fn)
scs = scs.loc[scs['AAR'] < 1].reset_index(drop=True) # remove rows where AAR = 1
# Load DEM
dem_fn = glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'DEMs', '*.tif'))[0]
dem = rxr.open_rasterio(dem_fn).isel(band=0)
dem = xr.where((dem < -1e3) | (dem > 1e4), np.nan, dem)
dem = dem.rio.write_crs("EPSG:4326")
# Load AOI
aoi_fn = os.path.join(scm_path, 'study-sites', rgi_id, 'AOIs', f"{rgi_id}_outline.shp")
aoi = gpd.read_file(aoi_fn)
# Clip DEM to AOI
dem = dem.rio.clip(aoi.geometry)
# Choose an image with some masking and relatively low transient AAR for demonstration
sc = scs.loc[(scs['datetime']=='2019-07-03T20:18:52') & (scs['source']=='Sentinel-2_SR')].reset_index(drop=True).iloc[0]
# sc = scs.loc[(scs['AAR'] < 0.5) & (scs['source']=='Sentinel-2_SR')].reset_index(drop=True).iloc[0]
classified_fn = glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'classified', 
                                       f"{sc['datetime'].replace('-', '').replace(':','')}_{rgi_id}_{sc['source']}_classified.nc"))[0]
classified = rxr.open_rasterio(classified_fn).squeeze()
classified = xr.where(classified==-9999, np.nan, classified)
classified = classified.rio.write_crs("EPSG:4326")
classified_utm = classified.rio.reproject(epsg_utm)
classified_utm = xr.where(classified_utm > 1e30, np.nan, classified_utm)

# Create binary snow image
snow_binary = xr.where((classified==1) | (classified==2), 1, 0)
snow_binary = xr.where(np.isnan(classified), np.nan, snow_binary) # re-insert no data values

# Regrid DEM to classified image grid
dem = dem.rio.reproject_match(classified)
dem = xr.where(dem > 1e4, np.nan, dem)
dem = dem.rio.write_crs("EPSG:4326")
dem_utm = dem.rio.reproject(epsg_utm)

# Calculate lower and upper bounds of snowline altitude
if sc['source']=='Landsat':
    dx=30
elif 'Sentinel-2' in sc['source']:
    dx=10
sla, sla_upper_bound, sla_lower_bound = calculate_sla_bounds(sc, dem, snow_binary, dx=dx, verbose=True)   
dem_utm = xr.where(dem_utm > 1e30, np.nan, dem_utm)
# Define colormap for classified images
cmap_dict = {"Snow": "#4eb3d3",  "Shadowed_snow": "#4eb3d3", "Ice": "#084081", "Rock": "#fe9929", "Water": "#969696"}
colors = []
for key in list(cmap_dict.keys()):
    color = list(matplotlib.colors.to_rgb(cmap_dict[key]))
    if key=='Rock':
        color += [0.5]
    colors.append(color)
cmap = matplotlib.colors.ListedColormap(colors)
cmap

In [ ]:
# Set up figure
fontsize=11
lw=1.5
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif': 'Arial'})
gs = matplotlib.gridspec.GridSpec(3,2, height_ratios=[1, 1.5, 1.5])
fig = plt.figure(figsize=(8,12))
ax = [fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1]),
      fig.add_subplot(gs[1,:]),
      fig.add_subplot(gs[2,:])]

# DEM
im = ax[0].imshow(dem_utm.data, cmap='terrain', 
                  extent=(np.min(dem_utm.x)/1e3, np.max(dem_utm.x)/1e3, np.min(dem_utm.y)/1e3, np.max(dem_utm.y)/1e3))
cbar = fig.colorbar(im, ax=ax[0], shrink=0.9, label='Elevation [m]')
ax[0].set_xlabel('Easting [km]')
ax[0].set_ylabel('Northing [km]')

# classified image
im = ax[1].imshow(classified_utm.data, cmap=cmap, clim=(0.5,5.5),
                  extent=(np.min(classified_utm.x)/1e3, np.max(classified_utm.x)/1e3, np.min(classified_utm.y)/1e3, np.max(classified_utm.y)/1e3))
cbar = fig.colorbar(im, ax=ax[1], shrink=0.9, ticks=[2, 3, 4, 5])
cbar.ax.set_yticklabels(['Snow', 'Ice/firn', 'Rock', 'Water'])
cbar.ax.set_ylim(1.5,5.5)
ax[1].set_yticklabels([])
ax[1].set_xlabel('Easting [km]')
ax[1].set_ylabel('')

# SLA contours
x_mesh, y_mesh = np.meshgrid(np.divide(dem_utm.x.data, 1e3), np.divide(dem_utm.y.data, 1e3))
for axis in ax[0:2]:
    axis.contour(x_mesh, y_mesh, dem_utm.data, levels=[sla], colors='k', linewidth=lw)

# histograms and bounds
bin_edges = np.arange(700, 1481, step=10)
bin_centers = (bin_edges[1:] + bin_edges[0:-1]) / 2
# all elevations
counts, _ = np.histogram(dem_utm.data, bins=bin_edges)
areas = counts * dx**2 / 1e6 # km^2
ax[2].bar(bin_centers, areas, width=bin_edges[1]-bin_edges[0], facecolor='gray', 
          edgecolor='k', alpha=0.5, linewidth=lw-1, label='All elevations')
# snow-covered elevations
dem_snow = xr.where(classified_utm==1, dem_utm, np.nan)
counts, _ = np.histogram(dem_snow.data, bins=bin_edges)
areas = counts * dx**2 / 1e6 # km^2
ax[2].bar(bin_centers, areas, width=bin_edges[1]-bin_edges[0], facecolor=cmap_dict['Snow'], 
          edgecolor='k', alpha=1.0, linewidth=lw-1, label='Snow-covered elevations')
ax[2].axvline(sla, color='k', linewidth=lw, label='Original SLA')
ax[2].axvline(sla_lower_bound, color='k', linestyle='--', linewidth=lw, label='SLA lower bound')
ax[2].axvline(sla_upper_bound, color='k', linestyle=':', linewidth=lw, label='SLA upper bound')
ax[2].text(850, 0.28, "Snow-covered area \nbelow SLA = 0.78 km$^2$", fontsize=9, ha='center',
           bbox=dict(facecolor='w', edgecolor='None', alpha=0.7))
ax[2].text(1350, 0.28, "Snow-free area \nabove SLA = 0.66 km$^2$", fontsize=9, ha='center')
ax[2].set_xlim(np.min(bin_edges)-20, np.max(bin_edges)+20)
ax[2].set_xlabel('Elevation [m]')
ax[2].set_ylabel('Area [km$^2$]')
ax[2].legend(loc='upper left')

# histogram of SLA ranges
ax[3].hist(sla_bounds_df['SLA_bounds_range_m'], bins=np.linspace(0, 1000, num=101), 
           facecolor='gray', alpha=0.9, edgecolor='k', linewidth=0.5)
range_median = np.nanmedian(sla_bounds_df['SLA_bounds_range_m'])
ax[3].axvline(range_median, color='k', linewidth=lw)
ax[3].text(160, 320, f"Median SLA range = {int(np.round(range_median, -1))} m", fontweight='bold')
ax[3].set_xlabel('Range of all SLA bounds [m]')
ax[3].set_ylabel('Counts')
ax[3].set_xlim(0, 600)
# add panel labels
labels = ['a', 'b', 'c', 'd']
for i, axis in enumerate(ax):
    if i < 2:
        xscale=0.85
        yscale=0.9
    else:
        xscale=0.95
        yscale=0.9
    axis.text(xscale, yscale, labels[i], transform=axis.transAxes, fontsize=fontsize+4, fontweight='bold')

fig.tight_layout()
plt.show()

# Save figure to file
fig_fn = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/glacier-snow-cover-analysis/figures/figS2_SLA_uncertainties.png'
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)



In [ ]:
import seaborn as sns
sns.histplot(sla_bounds_df, x='Zrange', y='SLA_bounds_range_m', cmap='rocket', bins=50, cbar=True)
plt.xlabel('Glacier elevation range [m]')
plt.ylabel('Range in SLA bounds [m]')